In [27]:
import acquire
import prepare
import explore
import model
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

# not included in final notebook
import os
import env
import numpy as np

# Project Overview

We will conduct an in depth analysis of Zillow property data from 2017. We will use exploratory analysis techniques to identify the key drivers of the assessed tax value for those properties, then use machine learning algorithms to create a model capable of predicting tax values based on features of the property. 

# Project Description

Property values have skyrocketed over the last two years. With such rapid changes in home values, predicting those values has become even more challenging than before. Since Zillow's estimate of home value is one of the primary drivers of its website traffic, having a reliable estimate is paramount. Any improvement we can make on the previous model will help us out-estimate our competitors and keep us at the top as the most trusted name in real estate technology. 

This project will analyze property attributes in relation to their 2017 assessed tax value, develop a model for predicting that value based on those attributes, and leave with recommendations for how to improve future predictions. 

# Project Goals

By improving upon the previous model, Zillow can enhance it's reputation for reliable property value estimates and better position itself in the real estate technology marketplace. 


# Acquisition

In this stage, we use a SQL query to access the Codeup MySQL database and join together all the relevant data using the following tables:

- properties_2017
- propertylandusetype
- predictions_2017

The data obtained includes all properties in the dataset which had a transaction in 2017.

The `zillow_2017_data` function from the `acquire` module performs this acquisition.

In [9]:
def zillow_2017_data():
    
    filename = 'zillow_2.csv'
    
    if os.path.exists(filename):
        print('Reading from local CSV...')
        return pd.read_csv(filename)
    
    url = env.get_db_url('zillow')
    sql = '''
            SELECT bedroomcnt, 
                    bathroomcnt, 
                    calculatedfinishedsquarefeet, 
                    taxvaluedollarcnt, 
                    yearbuilt, 
                    fips,
                    garagecarcnt,
                    garagetotalsqft
              FROM properties_2017
                JOIN propertylandusetype USING (propertylandusetypeid)
                JOIN predictions_2017 USING(parcelid)
              WHERE propertylandusedesc IN ("Single Family Residential", 
                                            "Inferred Single Family Residential")
                AND transactiondate LIKE "2017%%";
            '''
    
    print('No local file exists\nReading from SQL database...')
    df = pd.read_sql(sql, url)

    print('Saving to local CSV... ')
    df.to_csv(filename, index=False)
    
    return df

In [37]:
# acquire the data from the Codeup cloud database
df = zillow_2017_data()

Reading from local CSV...


# Preparation

In this stage, we clean and prepare the data for further exploration and modeling and encapsulate the relevant code in the prepare modele. 

The `prep_zillow_1` function accomplishes the following:

- checks for null values and removes all rows with null values if the number of null values is less than 5% the number of observations in the dataset. 
    - less than 5% of values is judged to not have a significant effect on a large dataset. Greater than 5% of values might require additional consideration. 
    - in this case we removed about 0.2% of the total observations due to null values
- renames columns for readability
- changes data types where appropriate (such as numerical codes into string values)
- adds a feature 'age', which represents years since the home was constructed

In [38]:
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,fips,garagecarcnt,garagetotalsqft
0,4.0,3.5,3100.0,1023282.0,1998.0,6059.0,2.0,633.0
1,2.0,1.0,1465.0,464000.0,1967.0,6111.0,1.0,0.0
2,3.0,2.0,1243.0,564778.0,1962.0,6059.0,2.0,440.0
3,4.0,3.0,2376.0,145143.0,1970.0,6037.0,NaN,NaN
4,4.0,3.0,2962.0,773303.0,1950.0,6037.0,NaN,NaN
...,...,...,...,...,...,...,...,...
52436,4.0,2.0,1633.0,346534.0,1962.0,6037.0,NaN,NaN
52437,2.0,2.0,1286.0,354621.0,1940.0,6037.0,NaN,NaN
52438,4.0,2.0,1612.0,67205.0,1964.0,6111.0,2.0,560.0
52439,3.0,1.0,1032.0,49546.0,1954.0,6037.0,NaN,NaN


In [39]:
# NEW:

df['garagetotalsqft'] = np.where(df.garagetotalsqft.isna(), 0, df.garagetotalsqft)

In [40]:
df

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,fips,garagecarcnt,garagetotalsqft
0,4.0,3.5,3100.0,1023282.0,1998.0,6059.0,2.0,633.0
1,2.0,1.0,1465.0,464000.0,1967.0,6111.0,1.0,0.0
2,3.0,2.0,1243.0,564778.0,1962.0,6059.0,2.0,440.0
3,4.0,3.0,2376.0,145143.0,1970.0,6037.0,NaN,0.0
4,4.0,3.0,2962.0,773303.0,1950.0,6037.0,NaN,0.0
...,...,...,...,...,...,...,...,...
52436,4.0,2.0,1633.0,346534.0,1962.0,6037.0,NaN,0.0
52437,2.0,2.0,1286.0,354621.0,1940.0,6037.0,NaN,0.0
52438,4.0,2.0,1612.0,67205.0,1964.0,6111.0,2.0,560.0
52439,3.0,1.0,1032.0,49546.0,1954.0,6037.0,NaN,0.0


In [11]:
# clean and prep the data using the steps outlined above
df = prepare.prep_zillow_1(df)

Number of null values > 5% length of df. Evaluate further before dropping nulls.


ValueError: cannot convert float NaN to integer

The `train_test_validate_split` function **Splits the data** into train, test, and validate samples

In [4]:
# split the data into three samples and print the sample sizes
train, test, validate = prepare.train_test_validate_split(df)

train	 n = 29298
test	 n = 10464
validate n = 12557


**Removing Outliers**

During initial exploration, it was discovered that extreme outliers exist in the categories of tax_value, bedrooms, bathrooms, sqft, and age. 

We judge that out model will generalize more effectively if it is focused on the typical (i.e. median) homes. Further, we expect that markets for luxury and historical homes (likely represented by these outliers) are considerably different than the general home market, and that an altogether separate model would be more useful for predicting the values of such homes. 

We will therefore drop outliers from this dataset. 

The `remove` outliers function calculates a range defining outliers based on the training sample, then removes values outside this range in all samples.

In [5]:
# remove outliers based on a range calculated from the training sample
train, validate, test = prepare.remove_outliers(train, validate, test, 1.5, ['bedrooms', 'bathrooms',
                                                                             'sqft', 'age', 'tax_value'])

train	 n = 26088
test	 n = 9241
validate n = 11144


# Modeling

In [13]:
# identify the name of the target feature
target = 'tax_value'

#### Scaling the data

Next, we will scale the values of each of our quantitative features so that they will be appropriately compared to each other within our models. In this case, we have used a Min-Max scaler, which will convert all values to within a range of 0 to 1. 

The `scale_zillow` function from the `prepare` module performs these scaling actions. 

In [14]:
# scale the data using sklearn MinMaxScaler()
train, validate, test = prepare.scale_zillow(train, validate, test, target)

#### Encoding the data

We will also encode each of our categorical features into a numerical data type that is interpretable by ML algorithms. 

The `encode_zillow` function from the prepare module performs these encoding actions. 

In [16]:
# encode the data using pandas' get_dummies
train, validate, test = prepare.encode_zillow(train, validate, test, target)

#### Establishing Baseline Performance

The code below sets up the necessary infrastructure for storing information about each of our models and their performance

In [17]:
# set the model_number at 0, to be changing with each modeling iteration
model_number = 0 
# create empty dataframes for storing model results and metadata about each model
model_info = pd.DataFrame()
model_results = pd.DataFrame()

Next we will establish our baseline predictions and evaluate their performance. 

To be able to determine whether our model gives useful predictions, we will compare it to a model that uses no features at all, and simply uses the mean value as a prediction for all observations. 

The `run_baseline` function from the `model` stores information about baseline predictions and their performance for later evaluation. 

In [18]:
# create baseline predictions and store information about their performance
model_number, model_info, model_results = model.run_baseline(train, validate, target, model_number, model_info, model_results)

#### Modeling: Ordinary Least Squares (OLS) Regression

In [19]:
# create OLS models and store information about their performance
model_number, model_info, model_results = model.run_OLS(train, validate, target, model_number, model_info, model_results)

## Model Evaluation

**Root Mean Squared Error (RMSE):** This is the metric we will use to evaluate the performance of our models. The RMSE tells us the standard deviation of the difference between our predictions and the actual values and therefore gives us an idea of how spread out the actual data points are around our regression line. Of the possible metrics for evaluating regression models, RMSE is easiest to interpret since it is expressed in units that are equivalent to our target variable (dollars in this case). 

Below, the model_info dataframe provides information about each model that we tested and the features that were included, and the model_results dataframe displays the RMSE for that model on each of the train and validate samples. 

In [20]:
# display the model_info dataframe
model_info

,model_number,model_type,features
0,baseline,baseline,NaN
1,1,OLS linear regression,"[scaled_bedrooms, scaled_bathrooms, scaled_sqft]"
2,2,OLS linear regression,"[scaled_bedrooms, scaled_bathrooms, scaled_sqf..."
3,3,OLS linear regression,"[scaled_bedrooms, scaled_bathrooms, scaled_sqf..."


In [21]:
# display the model_results dataframe
model_results

,model_number,sample_type,metric_type,score
0,baseline,train,RMSE,261710.496279
1,baseline,validate,RMSE,263821.689117
2,1,train,RMSE,228121.330688
3,1,validate,RMSE,227747.758899
4,2,train,RMSE,228099.897239
5,2,validate,RMSE,227734.292830
6,3,train,RMSE,226451.132843
7,3,validate,RMSE,225539.107024


Features used in Model 3:

In [22]:
# show the features used in Model 3
for feature in model_info[model_info.model_number == 3].features.values[0]:
    print(feature)

scaled_bedrooms
scaled_bathrooms
scaled_sqft
scaled_age
enc_fips_06059
enc_fips_06111


Model 3 is our highest performing model. It has an RMSE of 600,181 on the train sample and 51,358 on the validate sample. It uses number of bathrooms, number of bedrooms, square footage, age, and location by fips code as features. 

#### Modeling - Final Test

Now we will evaluate this model's performance on our test sample. This will approximate the expected performance of this model for predicting prices of additional properties in real-time.

The `final_test_model3` function peforms this test and displays the results.

In [23]:
# calculate the RMSE for model 3
model.final_test_model3(train, test, target)

Model 3 RMSE:  $226,607.30


# How did we do?

We expect that our model will typically predict a value that is within approximately \$549,000 of the actual value of the property. This large of an error makes this not an exceptionally useful model. However, this is 175,000 better than our baseline error of approximately 724,000. This indicates that we have successfully identified drivers of home value, and gives us a jumping off point for further analysis and modeling. 

### Key Findings:

We determined that the following factors are significant drivers of home value:
- number of bedrooms
- number of bathrooms
- square footage (finished area)

### Recommendations:

Zillow should continue to collect data regarding the number of bedrooms and bathrooms in a home, as well as the home's area in square feet. This data has been conclusively shown to assist in predicting home values. If using this analysis to decide which homes are worth investing in, an investor should lean towards homes with higher values in these categories, all other considerations being equal. 

### Next Steps: 

Given more time, I would examine additional features as drivers of home value. Some factors that I would expect to have significant influence include:

- whether or not a property has a pool or spa
- whether or not there is a garage on the property
- the size of the garage
- more granular information about location, such as zip code or neighborhood
    
These features could be explored directly, through visualization and statistical testing, or they could be identified through automated features selection techniques such as Recursive Feature Elimination. 

Additionally, since real estate markets are based heavily on location, I might expect models to perform better which individually focus on a distinct geographic area. 

The goal would be to produce a model that has an error small enough to be useful to someone intending to sell or purchase a single family home. 

In [24]:
train.head()

,bedrooms,bathrooms,sqft,tax_value,fips,age,scaled_bedrooms,scaled_bathrooms,scaled_sqft,scaled_age,enc_fips_06059,enc_fips_06111
13026,2.0,1.5,1012.0,216351.0,06059,45,0.000000,0.142857,0.232370,0.403670,1,0
27044,4.0,2.0,1352.0,98346.0,06037,62,0.666667,0.285714,0.324237,0.559633,0,0
30173,2.0,1.0,1401.0,268861.0,06037,77,0.000000,0.000000,0.337476,0.697248,0,0
3559,3.0,1.0,1225.0,432894.0,06037,94,0.333333,0.000000,0.289922,0.853211,0,0
33501,5.0,2.5,2424.0,676000.0,06111,49,1.000000,0.428571,0.613888,0.440367,0,1


In [25]:
train.shape

(26088, 12)